In [ ]:
import pandas as pd
import json
import matplotlib
import os
#os.chdir("/dccstor/arnaik_data/routing/model-improvement/data")
os.chdir("/Users/benjaminelder/ai4ba")
import pdb

In [ ]:
results_folder="results"
categories=["llms","agents"]
groups=["slot_filling","sequencing"] #,"rest"]
aggregation_filename="metrics_aggregation"
version='v3'
model_names_llms=[
    "llama-3-3-70b-instruct",
    "Llama-3.1-8B-Instruct",
    "Mixtral-8x22B-Instruct-v0.1",
    "DeepSeek-V3",
    "granite-3.1-8b-instruct",
    "hammer2.1-7b", 
    "watt-tool-8b", 
    "qwen2.5-7b-instruct", 
    "qwen2.5-72b-instruct",
    "gpt-4o",
]
model_names_nl2sql=["llama-3-3-70b-instruct","Llama-3.1-8B-Instruct","Mixtral-8x22B-Instruct-v0.1","DeepSeek-V3","granite-3.1-8b-instruct"]
model_names_agents=["llama-3-3-70b-instruct","Mixtral-8x22B-Instruct-v0.1","gpt-4o"]
INTENT_COL = "intent_list_metrics"
SLOT_COL = "slot_set_metrics"

In [ ]:
name_standardize_map={
    "llms":"llm",
    "agents":"agent",
    "llama-3-3-70b-instruct": "llama-3-3",
    "Llama-3.1-8B-Instruct": "llama-3-1",
    "Mixtral-8x22B-Instruct-v0.1": "mixtral-22b",
    "gpt-4o":"gpt-4o",
    "gpt4o":"gpt-4o",
}

In [ ]:
def get_metrics(data,category=None,group=None,model=None):
    if category == 'llms':
        # if group=='rest':
        #     pdb.set_trace()
        #     intent_metrics = data["aggregated_metrics_problem_level"]["compute_mode_meta_metrics"][INTENT_COL]["micro"][name_standardize_map[category]]
        #     slot_metrics = data["aggregated_metrics_problem_level"]["compute_mode_meta_metrics"][SLOT_COL]["micro"][name_standardize_map[category]]
        # else:
        intent_metrics = data["aggregated_metrics_problem_level"][name_standardize_map[category]][INTENT_COL]["micro"][model.lower()]
        slot_metrics = data["aggregated_metrics_problem_level"][name_standardize_map[category]][SLOT_COL]["micro"][model.lower()]
    elif category=='agents':
        if group=='rest':
            intent_metrics = data["aggregated_metrics_problem_level"]["compute_mode_meta_metrics"][INTENT_COL]["micro"]["llm"]
            slot_metrics = data["aggregated_metrics_problem_level"]["compute_mode_meta_metrics"][SLOT_COL]["micro"]["llm"]
        else:
            try:
                if model.lower() == "gpt-4o" and "gpt4o" in data["aggregated_metrics_problem_level"]["llm"][INTENT_COL]["micro"].keys():
                        intent_metrics = data["aggregated_metrics_problem_level"]["llm"][INTENT_COL]["micro"]["gpt4o"]
                        slot_metrics = data["aggregated_metrics_problem_level"]["llm"][SLOT_COL]["micro"]["gpt4o"]
                else:
                    intent_metrics = data["aggregated_metrics_problem_level"]["llm"][INTENT_COL]["micro"][model.lower()]
                    slot_metrics = data["aggregated_metrics_problem_level"]["llm"][SLOT_COL]["micro"][model.lower()]
            except:
                print("Trying model: ", model.lower())
                print("Only found models: ", data["aggregated_metrics_problem_level"]["llm"][INTENT_COL]["micro"].keys())
                raise
    return intent_metrics, slot_metrics

In [ ]:
def get_data(foldername,model_name=None):
    filename=None
    for file in os.listdir(foldername):
        if "metrics_aggregation" in file:
            if not model_name:
                filename=os.path.join(foldername,file)
                break
            elif model_name in file:
                filename=os.path.join(foldername,file)                
                break

    if not filename:
        print(f"No file named 'metrics_aggregation' in folder {foldername}")
        return None
    else:
        print(f"Processing ---- {filename}")
        data = json.load(open(filename, 'r'))  
        return data  


In [ ]:
def get_results_df(groups,model_names):
    row_names=[]
    col_names=[]
    for i in groups:
        for j in model_names:
            row_names.append(i+"=="+j)
    for i in ['intent','slot']:
        for j in ['precision','recall','F1']:
            col_names.append(i+"_"+j)
    return pd.DataFrame(columns=col_names, index=row_names)

In [ ]:
# Results for LLMs
category="llms"
results_llms=get_results_df(groups=groups,model_names=model_names_llms)
for group in groups:
    for model in model_names_llms:
        data=None
        if group =="rest":
            foldername=os.path.join(results_folder,str(category),str(group),str(version)+'/output/aggr/metrics_aggregation/')
            # foldername=os.path.join(results_folder,str(category),str(group),str(version)+'/output/aggr/'+str(model.lower())+'/metrics_aggregation/')
        else:
            foldername=os.path.join(results_folder,str(category),str(group))
        
        if os.path.isdir(foldername):
            data = get_data(foldername=foldername)
        else:
            print(f"{foldername} doesnot exist")
        if data:
            intent_metrics, slot_metrics = get_metrics(data,category=category,group=group,model=model)
            results_llms.loc[group+'=='+model]=[round(intent_metrics['precision'],2) if isinstance(intent_metrics['precision'],float) else intent_metrics['precision'],
                                                round(intent_metrics['recall'],2) if isinstance(intent_metrics['recall'],float) else intent_metrics['recall'],
                                                round(intent_metrics['f1'],2) if isinstance(intent_metrics['f1'],float) else intent_metrics['f1'],
                                                round(slot_metrics['precision'],2) if isinstance(slot_metrics['precision'],float) else slot_metrics['precision'],
                                                round(slot_metrics['recall'],2) if isinstance(slot_metrics['recall'],float) else slot_metrics['recall'],
                                                round(slot_metrics['f1'],2) if isinstance(slot_metrics['f1'],float) else slot_metrics['f1']]
results_llms = results_llms.add_prefix(f'{category}_')
print(f"-------------------------------{category}-------------------------------------")
print(results_llms)
print("------------------------------------------------------------------------------")

In [ ]:
results_llms

In [ ]:
## TODO: Win Rates for LLMs
category="llms"
results_llms["llms_win_rate"] = None
for group in groups:
    if group == "rest":
        filename = os.path.join(results_folder,str(category),str(group),str(version),"output/"+str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))        
    else:
        filename = os.path.join(results_folder,str(category),str(group),str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))
    data =  {k.lower(): v for k, v in data.items()}
    for model in model_names_llms:
        if model.lower() in data.keys():
            win_rate=data[model.lower()]["total_micro"]["win_rate"]
            results_llms.loc[str(group)+"=="+str(model),"llms_win_rate"] = round(win_rate,2) if isinstance(win_rate,float) else win_rate
        else:
            results_llms.loc[str(group)+"=="+str(model),"llms_win_rate"]="None"

In [ ]:
results_llms

In [ ]:
# Results for Agents
category="agents"
results_agents=get_results_df(groups=groups,model_names=model_names_agents)
for group in groups:
    for model in model_names_agents:
        data=None
        if group =="rest":
            foldername=os.path.join(results_folder,str(category),str(group),str(version)+'/aggr/'+str(name_standardize_map[model])+'/metrics_aggregation/')
            if os.path.isdir(foldername):
                data = get_data(foldername=foldername)
            else:
                print(f"{foldername} doesnot exist")
        else:
            foldername=os.path.join(results_folder,str(category),str(group))
            data = get_data(foldername=foldername)
        
        if data:
            intent_metrics, slot_metrics = get_metrics(data,category=category,group=group,model=model)
            results_agents.loc[group+'=='+model]=[round(intent_metrics['precision'],2) if isinstance(intent_metrics['precision'],float) else intent_metrics['precision'],
                                                round(intent_metrics['recall'],2) if isinstance(intent_metrics['recall'],float) else intent_metrics['recall'],
                                                round(intent_metrics['f1'],2) if isinstance(intent_metrics['f1'],float) else intent_metrics['f1'],
                                                round(slot_metrics['precision'],2) if isinstance(slot_metrics['precision'],float) else slot_metrics['precision'],
                                                round(slot_metrics['recall'],2) if isinstance(slot_metrics['recall'],float) else slot_metrics['recall'],
                                                round(slot_metrics['f1'],2) if isinstance(slot_metrics['f1'],float) else slot_metrics['f1']]
results_agents = results_agents.add_prefix(f'{category}_')
print(f"-------------------------------{category}-------------------------------------")
print(results_agents)
print("------------------------------------------------------------------------------")

In [ ]:
"""
## Win Rates for Agents
category="agents"
results_agents["agents_win_rate"] = None
for group in groups:
    if group == "rest":
        filename = os.path.join(results_folder,str(category),str(group),str(version),str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))        
    else:
        filename = os.path.join(results_folder,str(category),str(group),str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))
        if "gpt4o" in data.keys():
            data["gpt-4o"] = data["gpt4o"]
    for model in model_names_agents:
        if (name_standardize_map[model] in data.keys()):
            win_rate=data[name_standardize_map[model]]["total_micro"]["win_rate"]
            results_agents.loc[str(group)+"=="+str(model),"agents_win_rate"] = round(win_rate,2) if isinstance(win_rate,float) else win_rate
        else:
            results_agents.loc[str(group)+"=="+str(model),"agents_win_rate"]="None"
"""
## Win Rates for Agents
category="agents"
results_agents["agents_win_rate"] = None
for group in groups:
    if group == "rest":
        filename = os.path.join(results_folder,str(category),str(group),str(version),str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))        
    else:
        filename = os.path.join(results_folder,str(category),str(group),str(group)+"_win_rates.json")
        data=json.load(open(filename, 'r'))
        if "gpt4o" in data.keys():
            data["gpt-4o"] = data["gpt4o"]
    for model in model_names_agents:
        if (name_standardize_map[model] in data.keys()):
            win_rate=data[name_standardize_map[model]]["total_micro"]["win_rate"]
            results_agents.loc[str(group)+"=="+str(model),"agents_win_rate"] = round(win_rate,2)
            results_agents.loc[str(group)+"=="+str(model),"base_llm_win_rate"] = results_llms.loc[str(group)+"=="+str(model),"llms_win_rate"]
            tao_length=data[name_standardize_map[model]]["total_micro"].get("avg_loops", -1)
            results_agents.loc[str(group)+"=="+str(model),"agents_tao_length"] = round(tao_length,2)
        else:
            results_agents.loc[str(group)+"=="+str(model),"agents_win_rate"]="None"

In [ ]:
results_agents

In [ ]:
## Add win rates
results_llms = results_llms.fillna("None")
results_agents = results_agents.fillna("None")

In [ ]:
# Merged Results
results = results_llms.join(results_agents).fillna("-")
results=results.replace("None","n/a")

In [ ]:
# ## Add NL2SQL win rates
# category="nl2sql"
# results["nl2sql_win_rate"]=None
# for group in groups:
#     filename = os.path.join(results_folder,str(category),str(group),"metrics_aggregation.json")
#     data=json.load(open(filename, 'r'))
#     for model in model_names_nl2sql:
#         win_rate=data[model]["answer_rate"]
#         print(model,win_rate)
#         results.loc[str(group)+"=="+str(model),"nl2sql_win_rate"] = round(win_rate,2) if isinstance(win_rate,float) else win_rate


In [ ]:
results_llms

In [ ]:
results_agents

In [ ]:
results

In [ ]:
## TODO : Fix based on final decision
naming_map={
    "slot_filling":"SLOT-BIRD",
    "sequencing":"SEL-BIRD",
    "rest":"REST-BIRD",
    "llama-3-3-70b-instruct":"Llama-3-3-70b-Inst",
    "Llama-3.1-8B-Instruct":"Llama-3.1-8B-Inst",
    "Mixtral-8x22B-Instruct-v0.1":"Mixtral-8x22B-Inst",
    "DeepSeek-V3":"DeepSeek-V3",
    "granite-3.1-8b-instruct":"Granite-3.1-8b-Inst",
    "hammer2.1-7b":"Hammer2.1-7b",
    "qwen2.5-7b-instruct":"Qwen2.5-7b-Inst",
    "watt-tool-8b":"Watt-tool-8b", 
    'qwen2.5-72b-instruct':"Qwen2.5-72b-Inst",
    "gpt-4o":"gpt-4o",
    }
num='{1}'
type='{c}'
val="{}"
category=""
for idx, row in enumerate(results_llms.iterrows()):
    model = naming_map[row[0].split("==")[1]]
    row_text=f" & {model}"
    if category != naming_map[row[0].split("==")[0]]:
        category=naming_map[row[0].split("==")[0]]
        print(f"-------------------------------------------------{category}-------------------------------------------------")
    #     val="{"+category+"}"
    #     # row_text=f"\multicolumn{num}{type}{val} & {model}"
    #     row_text=f"{model}"        
    # else:
    #     val="{}"
    #     # row_text=f"\multicolumn{num}{type}{val} & {model}"
    #     row_text=f"{model}"        
    
    for i, item in enumerate(row[1]):
        item = "{"+str(item)+"}"
        row_text+= f" & {item} "
        if i == (len(row[1])-1):
            row_text+="\\\\"
    print(row_text)

In [ ]:
## Agents table

## TODO : Fix based on final decision
naming_map={
    "slot_filling":"SLOT-BIRD",
    "sequencing":"SEL-BIRD",
    "rest":"REST-BIRD",
    "llama-3-3-70b-instruct":"Llama-3-3-70b-Inst",
    "Llama-3.1-8B-Instruct":"Llama-3.1-8B-Inst",
    "Mixtral-8x22B-Instruct-v0.1":"Mixtral-8x22B-Inst",
    "DeepSeek-V3":"DeepSeek-V3",
    "granite-3.1-8b-instruct":"Granite-3.1-8b-Inst",
    "hammer2.1-7b":"Hammer2.1-7b",
    "qwen2.5-7b-instruct":"Qwen2.5-7b-Inst",
    "watt-tool-8b":"Watt-tool-8b", 
    'qwen2.5-72b-instruct':"Qwen2.5-72b-Inst",
    "gpt-4o":"gpt-4o",
    }
num='{1}'
type='{c}'
val="{}"
category=""
results_agents = results_agents[['base_llm_win_rate', 'agents_win_rate', 'agents_tao_length']]
for idx, row in enumerate(results_agents.iterrows()):
    model = naming_map[row[0].split("==")[1]]
    row_text=f"{model}"
    if category != naming_map[row[0].split("==")[0]]:
        category=naming_map[row[0].split("==")[0]]
        print(f"-------------------------------------------------{category}-------------------------------------------------")
    #     val="{"+category+"}"
    #     # row_text=f"\multicolumn{num}{type}{val} & {model}"
    #     row_text=f"{model}"        
    # else:
    #     val="{}"
    #     # row_text=f"\multicolumn{num}{type}{val} & {model}"
    #     row_text=f"{model}"        
    
    for i, item in enumerate(row[1]):
        item = "{"+str(item)+"}"
        row_text+= f" & {item} "
        if i == (len(row[1])-1):
            row_text+="\\\\"
    print(row_text)

In [ ]:
# \begin{table}[]
# \scriptsize
# \begin{tabular}{lcllllllllllllll}
#                               & \multicolumn{1}{l}{} & \multicolumn{7}{c}{Direct Invocation}                                                                                                                                          & \multicolumn{7}{c}{ReACT Agent}                                                                                                                            \\
# \multicolumn{1}{c}{Dataset}   & Model                & \multicolumn{3}{c}{Intent}                                             & \multicolumn{3}{c}{Slot}                                               & \multicolumn{1}{c}{Win Rate} & \multicolumn{3}{c}{Intent}                                             & \multicolumn{3}{c}{Slot}                                               & Win Rate \\
# \multicolumn{1}{c}{}          &                      & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{F1} & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{F1} &                              & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{F1} & \multicolumn{1}{c}{P} & \multicolumn{1}{c}{R} & \multicolumn{1}{c}{F1} &          \\
# \multicolumn{1}{c}{SLOT-BIRD} & M1                   & \multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{1}{c}{}   & \multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{1}{c}{}   &                              &                       &                       &                        &                       &                       &                        &          \\
# \multicolumn{1}{c}{}          & M2                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
# \multicolumn{1}{c}{}          & M3                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
# SEL-BIRD                      & M1                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
#                               & M2                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
#                               & M3                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
# REST-BIRD                     & M1                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &          \\
#                               & M2                   &                       &                       &                        &                       &                       &                        &                              &                       &                       &                        &                       &                       &                        &         
# \end{tabular}
# \label{tab:main-results}
# \todo{Since we dont have all models as agents may need to break? }
# \caption{Performance of Models and ReACT Agents on our dataset}
# \end{table}